In [ ]:
import sys
sys.executable

'/usr/bin/python3'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!git clone https://github.com/nyu-mll/jiant.git
!cd jiant

Cloning into 'jiant'...
remote: Enumerating objects: 13826, done.
remote: Counting objects: 100% (129/129), done.
remote: Compressing objects: 100% (100/100), done.
remote: Total 13826 (delta 42), reused 76 (delta 26), pack-reused 13697
Receiving objects: 100% (13826/13826), 4.39 MiB | 15.23 MiB/s, done.
Resolving deltas: 100% (9648/9648), done.


In [ ]:
%%capture
!pip install -r jiant/requirements.txt
!pip install jiant

In [ ]:
tasks = [
    "superglue_broadcoverage_diagnostics", # Broadcoverage Diagnostics; Recognizing Textual Entailment
    "cb", # CommitmentBank
    "copa", # Choice of Plausible Alternatives
    "multirc", # Multi-Sentence Reading Comprehension
    "wic", # Words in Context
    "wsc", # The Winograd Schema Challenge
    "boolq", # BoolQ
    "record", # Reading Comprehension with Commonsense Reasoning
    "superglue_winogender_diagnostics", # Winogender Schema Diagnostics
    "rte"
]

In [ ]:
# Download Data
!python jiant/jiant/scripts/download_data/runscript.py \
    download \
    --tasks "superglue_winogender_diagnostics" \
    --output_path "/content/drive/MyDrive/superglue/tasks"

Downloading: 100% 10.4k/10.4k [00:00<00:00, 5.43MB/s]
Dataset super_glue downloaded and prepared to /root/.cache/huggingface/datasets/super_glue/axg/1.0.2/41d9edb3935257e1da4b7ce54cd90df0e8bb255a15e46cfe5cbc7e1c04f177de. Subsequent calls will reuse this data.
Downloaded and generated configs for 'superglue_winogender_diagnostics' (1/1)


In [ ]:
# Download Model
!python jiant/jiant/proj/main/export_model.py \
    --hf_pretrained_model_name_or_path "/content/drive/MyDrive/bert-english" \
    --output_base_path "/content/drive/MyDrive/superglue/models/bert-english"

In [ ]:
# Cache Data
!python jiant/jiant/proj/main/tokenize_and_cache.py \
    --task_config_path "/content/drive/MyDrive/superglue/tasks/configs/superglue_winogender_diagnostics_config.json" \
    --hf_pretrained_model_name_or_path "/content/drive/MyDrive/bert-english" \
    --output_dir "/content/drive/MyDrive/superglue/cache/bert/superglue_winogender_diagnostics" \
    --phases test \
    --max_seq_length 512 \
    --smart_truncate

SuperglueWinogenderDiagnosticsTask
  [test]: /content/drive/MyDrive/superglue/tasks/data/superglue_winogender_diagnostics/test.jsonl
Tokenizing: 100% 356/356 [00:00<00:00, 1055.63it/s]
Smart truncate data: 100% 356/356 [00:00<00:00, 100570.64it/s]


In [ ]:
# Create config
!python jiant/jiant/proj/main/scripts/configurator.py \
    SingleTaskConfigurator \
    /content/drive/MyDrive/superglue/runconfigs/bert_enwiki/superglue_winogender_diagnostics.json \
    --task_name superglue_winogender_diagnostics \
    --task_config_base_path /content/drive/MyDrive/superglue/tasks/configs \
    --task_cache_base_path /content/drive/MyDrive/superglue/cache/bert \
    --train_batch_size 16 \
    --eval_batch_multiplier 2 \
    --do_test 

In [ ]:
# Train model 
!python jiant/jiant/proj/main/runscript.py \
    run \
    --ZZsrc superglue/models/bert-english/config.json \
    --jiant_task_container_config_path  /content/drive/MyDrive/superglue/runconfigs/bert_enwiki/superglue_winogender_diagnostics.json \
    --learning_rate 1e-5 \
    --do_val \
    --do_save \
    --no_improvements_for_n_evals 3 \
    --output_dir /content/drive/MyDrive/superglue/runs/bert-english/superglue_winogender_diagnostics \
    --write_test_preds

In [ ]:
# Convert to RSG Format
!python jiant/jiant/scripts/benchmarks/benchmark_submission_formatter.py \
    --benchmark SUPERGLUE \
    --input_base_path /content/drive/MyDrive/superglue/runs/bert-english \
    --output_path /content/drive/MyDrive/superglue/preds/bert-english \
    --tasks copa